# Exercise on the value of unsupervised constructed features for training a classifier with few labeled examples:SOLUTION

To get unsupervised conStructed features of an image, we have used a pretrained CNN as feature extractor. For this purpose we pushed each image through a pretrained CNN and extracted the activations in the first fully connected layer. As pretrained CNN we use a VGG16 architecture that was trained on ImageNet data and was the second winner of the ImageNet competition in 2014.

In this manner we have got unsupervised constructed features for 1000 images of the MNIST data set and 1000 images of the CIFAR10 data set. In both data sets we have 10 distinguished classes. The data sets are balanced meaning we have 100 images per class. 

In the last exercise we have used 2D plots to check if the feature quality is good enough to lead to visible clusters corresoonding to the 10 classes in the data. For the MNISt data it is very easy to seperate the 10 digits. Therefore we will only continue with the more challenging CIFAR10 data.

As a second check on the quality of the feature representation of the CIFAR10 data, we will use once the pixel-features and once the VGG-features to train a classifier using only 100 labeled data (on average 10 per class). If the VGG-feature are indeed better, we would expect to achieve a better classifier when using the VGG-feature compared to the pixel feature.

a) Which accuracy would you expect for a classifier which cannot distinguish between the 10 classes and is only guessing?

**Solution: 10%**


b) Go through the code which is used to set-up, train, and evaluate a CNN classifier using the raw pixel features. Discuss your thoughts on the achieved accuracy (e.g. with your neighbor).

**Solution: The accuracy is with ~20% better then guessing but still very bad. However, this is not surprising since the resolution of the images are very low and it is alread by eye quite difficutl to distinguish between the classes. Moreover, we have  only very few training examples (only 10 per class), quite bad features (the raw pixel values) and a model with many parameters (~45k parameter).**

b) Now we use the unsupervised constructed VGG features. We want to check, if these VGG features are good enough to train a classifier with only few labeled data and still get a satisfying performance. For this purpose, please complet the code to set up a fully connected NN and run the provided subsequent code to train it and determine its accuracy on the test set. Compare it to the accuracy which we achieve with a RF. Discuss the results (e.g. with your neighbor).

**Solution: For code completion see below. The accuracy of the fcNN is with more than 55% much better than the accuray of the from scratch trained CNN which was 20%. This implies that the VGG-features are quite good and more informative than the raw pixel features. With the RF we do not achieve a better performance - it looks even worse than the fcNN. This is quite surprising since we would expect the fcNN to overfit the data since it has only 1000 training data and ~800 parameters. A possible reason might be that the features were constructed in the VGG model for a following fcNN classifier.**



### Imports

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from pylab import *


import time
import tensorflow as tf
tf.set_random_seed(1)

import keras
import sys
print ("Keras {} TF {} Python {}".format(keras.__version__, tf.__version__, sys.version_info))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Keras 2.1.4 TF 1.6.0 Python sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


Using TensorFlow backend.


### CIFAR Data preparation

In [2]:
#downlad cifar data
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
del [x_test,y_test]

In [3]:
#loop over each class label and sample 100 random images over each label and save the idx to subset
np.random.seed(seed=222)
idx=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_train))):
    idx=np.append(idx,np.random.choice(np.where((y_train[0:len(y_train)])==i)[0],100,replace=False))

x_train= x_train[idx]
y_train= y_train[idx]

In [4]:
print(x_train.shape)
print(y_train.shape)
print(np.unique(y_train,return_counts=True))

(1000, 32, 32, 3)
(1000, 1)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100]))


In [5]:
#make train vaild and test
#loop over each class label and sample 100 random images over each label and save the idx to subset
np.random.seed(seed=123)
idx_train=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_train))):
    idx_train=np.append(idx_train,np.random.choice(np.where((y_train[0:len(y_train)])==i)[0],10,replace=False))

x_train_new = x_train[idx_train]
y_train_new = y_train[idx_train]

In [6]:
x_test_new=(np.delete(x_train,idx_train,axis=0))
y_test_new=(np.delete(y_train,idx_train,axis=0))

In [7]:
np.random.seed(seed=127)
idx_vaild=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_test_new))):
    idx_vaild=np.append(idx_vaild,np.random.choice(np.where((y_test_new[0:len(y_test_new)])==i)[0],10,replace=False))

x_vaild_new = x_test_new[idx_vaild]
y_valid_new = y_test_new[idx_vaild]

In [8]:
x_test_new=(np.delete(x_test_new,idx_vaild,axis=0))
y_test_new=(np.delete(y_test_new,idx_vaild,axis=0))

In [9]:
x_train_new = np.reshape(x_train_new, (100,32,32,3))
x_vaild_new = np.reshape(x_vaild_new, (100,32,32,3))
x_test_new = np.reshape(x_test_new, (800,32,32,3))

In [10]:
print(np.unique(y_train_new,return_counts=True))
print(np.unique(y_valid_new,return_counts=True))
print(np.unique(y_test_new,return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([80, 80, 80, 80, 80, 80, 80, 80, 80, 80]))


In [11]:
from keras.utils.np_utils import to_categorical   

y_train_new=to_categorical(y_train_new,10)
y_valid_new=to_categorical(y_valid_new,10)
y_test_new=to_categorical(y_test_new,10)



print(x_train_new.shape)
print(y_train_new.shape)

print(x_vaild_new.shape)
print(y_valid_new.shape)

print(x_test_new.shape)
print(y_test_new.shape)

In [12]:
# center and standardize the data
X_mean = np.mean( x_train_new, axis = 0)
X_std = np.std( x_train_new, axis = 0)

x_train_new = (x_train_new - X_mean ) / (X_std + 0.0001)
x_vaild_new = (x_vaild_new - X_mean ) / (X_std + 0.0001)
x_test_new = (x_test_new - X_mean ) / (X_std + 0.0001)

### Setting up the the CNN classifier based on raw image data

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, Flatten


In [14]:
# here we define  hyperparameter of the NN
batch_size = 10
nb_classes = 10
nb_epoch = 30
img_rows, img_cols = 32, 32
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, 3)
pool_size = (2, 2)

In [20]:
model = Sequential()

model.add(Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution2D(8, kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(16, kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(16,kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(40))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 8)         224       
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 8)         32        
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 8)         584       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 8)         32        
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 8)         0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 8)         0         
__________

In [22]:
history=model.fit(x_train_new, y_train_new, 
                  batch_size=10, 
                  epochs=30,
                  verbose=2, 
                  validation_data=(x_vaild_new, y_valid_new),shuffle=True)

Train on 100 samples, validate on 100 samples
Epoch 1/30
 - 2s - loss: 2.7205 - acc: 0.1200 - val_loss: 2.8294 - val_acc: 0.1400
Epoch 2/30
 - 1s - loss: 2.0518 - acc: 0.3000 - val_loss: 2.6306 - val_acc: 0.1500
Epoch 3/30
 - 1s - loss: 1.7144 - acc: 0.4300 - val_loss: 2.4216 - val_acc: 0.1300
Epoch 4/30
 - 1s - loss: 1.6136 - acc: 0.4300 - val_loss: 2.3575 - val_acc: 0.1200
Epoch 5/30
 - 1s - loss: 1.3343 - acc: 0.6100 - val_loss: 2.3813 - val_acc: 0.1700
Epoch 6/30
 - 1s - loss: 1.2349 - acc: 0.6700 - val_loss: 2.3909 - val_acc: 0.2200
Epoch 7/30
 - 1s - loss: 1.2331 - acc: 0.6600 - val_loss: 2.4204 - val_acc: 0.1700
Epoch 8/30
 - 1s - loss: 1.1278 - acc: 0.7500 - val_loss: 2.4020 - val_acc: 0.1600
Epoch 9/30
 - 1s - loss: 0.8816 - acc: 0.8100 - val_loss: 2.3726 - val_acc: 0.1900
Epoch 10/30
 - 1s - loss: 0.8182 - acc: 0.8400 - val_loss: 2.3940 - val_acc: 0.2300
Epoch 11/30
 - 1s - loss: 0.7492 - acc: 0.8500 - val_loss: 2.3765 - val_acc: 0.2200
Epoch 12/30
 - 1s - loss: 0.7999 - acc:

### Evaluation of the CNN classifier that was trained on raw image data

In [23]:
from sklearn.metrics import confusion_matrix
pred=model.predict(x_test_new)
print(confusion_matrix(np.argmax(y_test_new,axis=1),np.argmax(pred,axis=1)))
print("Acc = " ,np.sum(np.argmax(y_test_new,axis=1)==np.argmax(pred,axis=1))/len(y_test_new))


[[28  2  9  2  8  3  3  9 16  0]
 [11  5 11  2  9  9  1  2 16 14]
 [ 8  2 17  2 17  7 13  4  8  2]
 [ 4  1 20  7 10 14  9  5  6  4]
 [ 5  1 16  6 14  3 20 10  4  1]
 [ 3  2 13  5 11 13 10 15  4  4]
 [ 2  1 14  5 10 17 23  2  3  3]
 [ 2  0  9  2 22  9  5 25  3  3]
 [21  3  5  1 10  6  1  5 23  5]
 [ 5  7  9  1 10 13  2  3 15 15]]
Acc =  0.2125


### Getting the VGG features for CIFAR

In [24]:
# Downloading embeddings
import urllib
import os
if not os.path.isfile('cifar_EMB_1000.npz'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/si287al91c1ls0d/cifar_EMB_1000.npz?dl=1",
    "cifar_EMB_1000.npz")
%ls -hl cifar_EMB_1000.npz

-rw-r--r-- 1 root root 18M Mar 29 12:35 cifar_EMB_1000.npz


In [25]:
Data=np.load("cifar_EMB_1000.npz")
vgg_features_cifar = Data["arr_0"]

In [26]:
vgg_features_cifar_train = vgg_features_cifar[idx_train]
vgg_features_cifar_test=(np.delete(vgg_features_cifar,idx_train,axis=0))
vgg_features_cifar_valid = vgg_features_cifar_test[idx_vaild]
vgg_features_cifar_test=(np.delete(vgg_features_cifar_test,idx_vaild,axis=0))


In [27]:
print(vgg_features_cifar_train.shape)
print(vgg_features_cifar_valid.shape)
print(vgg_features_cifar_test.shape)

(100, 4096)
(100, 4096)
(800, 4096)


### Setting up the the CNN classifier based on VGG feature

In [28]:
model = Sequential()
model.add(Dense(200,batch_input_shape=(None, 4096)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(200,input_shape=(None,4096)))

#### we still need to add the last layers to get the predictions on the 10 classes
### your code here

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

####### end of your code ######


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 200)               819400    
_________________________________________________________________
batch_normalization_11 (Batc (None, 200)               800       
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
activation_13 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                2010      
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
Total para

In [30]:
history=model.fit(vgg_features_cifar_train, y_train_new, 
                  batch_size=10, 
                  epochs=20,
                  verbose=2, 
                  validation_data=(vgg_features_cifar_valid, y_valid_new),shuffle=True)

Train on 100 samples, validate on 100 samples
Epoch 1/20
 - 1s - loss: 2.3676 - acc: 0.2700 - val_loss: 2.5823 - val_acc: 0.3700
Epoch 2/20
 - 0s - loss: 0.8896 - acc: 0.7200 - val_loss: 2.1334 - val_acc: 0.4300
Epoch 3/20
 - 0s - loss: 0.5962 - acc: 0.7800 - val_loss: 1.6625 - val_acc: 0.5100
Epoch 4/20
 - 0s - loss: 0.4425 - acc: 0.8900 - val_loss: 1.7256 - val_acc: 0.5000
Epoch 5/20
 - 0s - loss: 0.4135 - acc: 0.8800 - val_loss: 1.7016 - val_acc: 0.5200
Epoch 6/20
 - 0s - loss: 0.2874 - acc: 0.9100 - val_loss: 1.6401 - val_acc: 0.5400
Epoch 7/20
 - 0s - loss: 0.2953 - acc: 0.9000 - val_loss: 1.5616 - val_acc: 0.5400
Epoch 8/20
 - 0s - loss: 0.2427 - acc: 0.9200 - val_loss: 1.5743 - val_acc: 0.5000
Epoch 9/20
 - 0s - loss: 0.2008 - acc: 0.9500 - val_loss: 1.6489 - val_acc: 0.5200
Epoch 10/20
 - 0s - loss: 0.1446 - acc: 0.9500 - val_loss: 1.6543 - val_acc: 0.4900
Epoch 11/20
 - 0s - loss: 0.1986 - acc: 0.9300 - val_loss: 1.7274 - val_acc: 0.4900
Epoch 12/20
 - 0s - loss: 0.0897 - acc:

### Evaluation of the CNN classifier that was trained on VGG features

In [31]:
pred=model.predict(vgg_features_cifar_test)

#### we now want to get the confusion matrix for the predictions on the test data
### your code here

print(confusion_matrix(np.argmax(y_test_new,axis=1),np.argmax(pred,axis=1)))
print("Acc = " ,np.sum(np.argmax(y_test_new,axis=1)==np.argmax(pred,axis=1))/len(y_test_new))

########## end of your code ###############################


[[49  1  2  1  0  0  1  2 20  4]
 [ 6 55  0  0  0  1  1  3  4 10]
 [14  0 31  1 13  8  5  1  6  1]
 [ 4  0  1 16  1 23 18  2  9  6]
 [ 4  0  8  2 37  3  4 17  5  0]
 [ 2  0  0 11  6 46  6  8  0  1]
 [ 5  0 26  2  6  2 37  0  2  0]
 [ 3  0  1  3 13  4  2 52  1  1]
 [11  1  0  0  0  0  0  0 67  1]
 [ 4 14  0  0  1  1  0  0  5 55]]
Acc =  0.55625


### Baseline: use VGG feature to train a Random Forest model

In [35]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(vgg_features_cifar_train,np.argmax(y_train_new, axis=1))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
from sklearn.metrics import confusion_matrix
pred=clf.predict(vgg_features_cifar_test)
print(confusion_matrix(np.argmax(y_test_new, axis=1), pred))
np.sum(pred==np.argmax(y_test_new, axis=1))/len(np.argmax(y_test_new, axis=1))


[[30  5  8  2  2  1  2  6 21  3]
 [ 7 40  0  0  3  8  1  1  8 12]
 [ 6  0 32 12  7  5  2 11  3  2]
 [ 3  8 18 20  3  8  7  6  5  2]
 [ 4  2 25 10 15  2  6 11  4  1]
 [ 3  2 10 16  3 39  2  4  1  0]
 [ 7  2 31  7  6  9  7  6  4  1]
 [ 5  2 11  5 13  7  1 31  4  1]
 [10  7  8  1  2  5  1  4 38  4]
 [ 4 16  1  1  1  4  1  2  8 42]]


0.3675